# Generate TIFF Reference File

In [1]:
import glob
import logging
from tempfile import TemporaryDirectory

import dask
import fsspec
import numpy as np
import pandas as pd
import rioxarray
import s3fs
import ujson
import xarray as xr
import xrefcoord  # noqa
from distributed import Client
from kerchunk.combine import MultiZarrToZarr
from kerchunk.tiff import tiff_to_zarr
from tqdm import tqdm

In [2]:
# # URL pointing to a single GeoTIFF file
# url = "s3://sentinel-cogs/sentinel-s2-l2a-cogs/44/P/MT/2022/5/S2A_44PMT_20220505_0_L2A/B12.tif"

# # Initialize a s3 filesystem
# fs = s3fs.S3FileSystem(anon=True)

# xds = rioxarray.open_rasterio(fs.open(url))

In [3]:
# xds.plot(robust=True)

In [4]:
# fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)

# files_paths = fs_read.glob(
# "s3://sentinel-cogs/sentinel-s2-l2a-cogs/44/P/MT/2022/5/*/B12.tif"
# )
# file_pattern = sorted(["s3://" + f for f in files_paths])

# so = dict(mode="rb", anon=True, default_fill_cache=False, default_cache_type="first")


# td = TemporaryDirectory()
# temp_dir = td.name

# client = Client(n_workers=8, silence_logs=logging.ERROR)
# client


# def generate_json_reference(fil, output_dir: str):
#     tiff_chunks = tiff_to_zarr(fil, remote_options={"protocol": "s3", "anon": True})
#     fname = fil.split("/")[-2].split('_')[2] + '.json'

#     outf = f"{output_dir}/{fname}.json"
#     with open(outf, "wb") as f:
#         f.write(ujson.dumps(tiff_chunks).encode())
#     return outf


# tasks = [dask.delayed(generate_json_reference)(fil, temp_dir) for fil in file_pattern]

# dask.compute(tasks)

# ref_files = sorted(glob.iglob(f"{temp_dir}/*.json"))


# def fn_to_time(index, fs, var, fn):
#     import datetime
#     import re

#     subst = fn.split("/")[-1].split(".json")[0]
#     return datetime.datetime.strptime(subst, "%Y%m%d")


# mzz = MultiZarrToZarr(
#     path=ref_files,
#     indicts=ref_files,
#     remote_protocol="s3",
#     remote_options={"anon": True},
#     coo_map={"time": fn_to_time},
#     coo_dtypes={"time": np.dtype("M8[s]")},
#     concat_dims=["time"],
#     identical_dims=["x", "y"],
# )

# multi_kerchunk = mzz.translate()

# output_fname = "references/sentinel_b12.json"
# with open(f"{output_fname}", "wb") as f:
#     f.write(ujson.dumps(multi_kerchunk).encode())

In [5]:
import fsspec
import rioxarray
import s3fs
import ujson
import xarray as xr
import xrefcoord  # noqa
from kerchunk.tiff import tiff_to_zarr

from xrefcoord.coords import _get_shape, _generate_coords

import imagecodecs.numcodecs

imagecodecs.numcodecs.register_codecs()

In [6]:
fs = fsspec.filesystem(
    "reference",
    fo="references/sentinel_b12.json",
    remote_protocol="s3",
    remote_options={"anon": True},
    skip_instance_cache=True,
)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr")

/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/ipykernel_89753/4145011781.py:9: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  ds = xr.open_dataset(m, engine="zarr")


In [25]:
sds = ds[["0"]]
sds.attrs = ds.attrs
sds.chunk({"time": -1})

<xarray.Dataset>
Dimensions:  (time: 6, Y: 5490, X: 5490)
Coordinates:
  * time     (time) datetime64[s] 2022-05-05 2022-05-10 ... 2022-05-30
Dimensions without coordinates: Y, X
Data variables:
    0        (time, Y, X) float32 dask.array<chunksize=(6, 5490, 5490), meta=np.ndarray>
Attributes: (12/14)
    multiscales:             [{'datasets': [{'path': '0'}, {'path': '1'}, {'p...
    OVR_RESAMPLING_ALG:      AVERAGE
    KeyDirectoryVersion:     1
    KeyRevision:             1
    KeyRevisionMinor:        0
    GTModelTypeGeoKey:       1
    ...                      ...
    GeogCitationGeoKey:      WGS 84
    GeogAngularUnitsGeoKey:  9102
    ProjectedCSTypeGeoKey:   32644
    ProjLinearUnitsGeoKey:   9001
    ModelPixelScale:         [20.0, 20.0, 0.0]
    ModelTiepoint:           [0.0, 0.0, 0.0, 399960.0, 1300020.0, 0.0]

In [26]:
sds

<xarray.Dataset>
Dimensions:  (time: 6, Y: 5490, X: 5490)
Coordinates:
  * time     (time) datetime64[s] 2022-05-05 2022-05-10 ... 2022-05-30
Dimensions without coordinates: Y, X
Data variables:
    0        (time, Y, X) float32 ...
Attributes: (12/14)
    multiscales:             [{'datasets': [{'path': '0'}, {'path': '1'}, {'p...
    OVR_RESAMPLING_ALG:      AVERAGE
    KeyDirectoryVersion:     1
    KeyRevision:             1
    KeyRevisionMinor:        0
    GTModelTypeGeoKey:       1
    ...                      ...
    GeogCitationGeoKey:      WGS 84
    GeogAngularUnitsGeoKey:  9102
    ProjectedCSTypeGeoKey:   32644
    ProjLinearUnitsGeoKey:   9001
    ModelPixelScale:         [20.0, 20.0, 0.0]
    ModelTiepoint:           [0.0, 0.0, 0.0, 399960.0, 1300020.0, 0.0]

In [ ]:
# fs = fsspec.filesystem(
#     "reference",
#     fo="references/sentinel_b12.json",
#     remote_protocol="s3",
#     remote_options={"anon": True},
#     skip_instance_cache=True,
# )
# m = fs.get_mapper("")
# ds = xr.open_dataset(m, engine="zarr", consolidated=False)

In [ ]:
# import dask.array as da

# def gen_xcoords(ds):
#     shape = ds[list(ds.variables)[0]].shape
#     xscale = ds.attrs["ModelPixelScale"][0]
#     x0 = ds.attrs["ModelTiepoint"][3]
#     return da.arange(shape[-1]) * xscale + x0 + xscale / 2

# def gen_ycoords(ds):
#     shape = ds[list(ds.variables)[0]].shape
#     yscale = ds.attrs["ModelPixelScale"][1]
#     y0 = ds.attrs["ModelTiepoint"][4]
#     return da.arange(shape[-2]) * -yscale + y0 - yscale / 2

In [ ]:
# cds = ds.xref.generate_ds_coords()

In [23]:
cds = _generate_coords(sds, x_dim_name="x", y_dim_name="y")

numcodec 'imagecodecs_aec' already registered
numcodec 'imagecodecs_apng' already registered
numcodec 'imagecodecs_avif' already registered
numcodec 'imagecodecs_bitorder' already registered
numcodec 'imagecodecs_bitshuffle' already registered
numcodec 'imagecodecs_blosc' already registered
numcodec 'imagecodecs_blosc2' already registered
numcodec 'imagecodecs_brotli' already registered
numcodec 'imagecodecs_byteshuffle' already registered
numcodec 'imagecodecs_bz2' already registered
numcodec 'imagecodecs_cms' already registered
numcodec 'imagecodecs_deflate' already registered
numcodec 'imagecodecs_delta' already registered
numcodec 'imagecodecs_float24' already registered
numcodec 'imagecodecs_floatpred' already registered
numcodec 'imagecodecs_gif' already registered
numcodec 'imagecodecs_heif' already registered
numcodec 'imagecodecs_jetraw' already registered
numcodec 'imagecodecs_jpeg' already registered
numcodec 'imagecodecs_jpeg2k' already registered
numcodec 'imagecodecs_jpeg

<xarray.Dataset>
Dimensions:  (time: 6, Y: 5490, X: 5490, x: 5490, y: 5490)
Coordinates:
  * time     (time) datetime64[s] 2022-05-05 2022-05-10 ... 2022-05-30
  * x        (x) float64 4e+05 4e+05 4e+05 ... 5.097e+05 5.097e+05 5.098e+05
  * y        (y) float64 1.3e+06 1.3e+06 1.3e+06 ... 1.19e+06 1.19e+06 1.19e+06
Dimensions without coordinates: Y, X
Data variables:
    0        (time, Y, X) float32 ...
Attributes: (12/14)
    multiscales:             [{'datasets': [{'path': '0'}, {'path': '1'}, {'p...
    OVR_RESAMPLING_ALG:      AVERAGE
    KeyDirectoryVersion:     1
    KeyRevision:             1
    KeyRevisionMinor:        0
    GTModelTypeGeoKey:       1
    ...                      ...
    GeogCitationGeoKey:      WGS 84
    GeogAngularUnitsGeoKey:  9102
    ProjectedCSTypeGeoKey:   32644
    ProjLinearUnitsGeoKey:   9001
    ModelPixelScale:         [20.0, 20.0, 0.0]
    ModelTiepoint:           [0.0, 0.0, 0.0, 399960.0, 1300020.0, 0.0]

<xarray.Dataset>
Dimensions:  (time: 6, Y: 5490, X: 5490, x: 5490, y: 5490)
Coordinates:
  * time     (time) datetime64[s] 2022-05-05 2022-05-10 ... 2022-05-30
  * x        (x) float64 4e+05 4e+05 4e+05 ... 5.097e+05 5.097e+05 5.098e+05
  * y        (y) float64 1.3e+06 1.3e+06 1.3e+06 ... 1.19e+06 1.19e+06 1.19e+06
Dimensions without coordinates: Y, X
Data variables:
    0        (time, Y, X) float32 dask.array<chunksize=(6, 5490, 5490), meta=np.ndarray>
Attributes: (12/14)
    multiscales:             [{'datasets': [{'path': '0'}, {'path': '1'}, {'p...
    OVR_RESAMPLING_ALG:      AVERAGE
    KeyDirectoryVersion:     1
    KeyRevision:             1
    KeyRevisionMinor:        0
    GTModelTypeGeoKey:       1
    ...                      ...
    GeogCitationGeoKey:      WGS 84
    GeogAngularUnitsGeoKey:  9102
    ProjectedCSTypeGeoKey:   32644
    ProjLinearUnitsGeoKey:   9001
    ModelPixelScale:         [20.0, 20.0, 0.0]
    ModelTiepoint:           [0.0, 0.0, 0.0, 399960.0, 1300020.0, 0.0]

In [8]:
cds

<xarray.Dataset>
Dimensions:  (time: 6, Y: 5490, X: 5490, Y1: 2745, X1: 2745, Y2: 1373,
              X2: 1373, Y3: 687, X3: 687, Y4: 344, X4: 344, x: 5490, y: 5490)
Coordinates:
  * time     (time) datetime64[s] 2022-05-05 2022-05-10 ... 2022-05-30
  * x        (x) float64 4e+05 4e+05 4e+05 ... 5.097e+05 5.097e+05 5.098e+05
  * y        (y) float64 1.3e+06 1.3e+06 1.3e+06 ... 1.19e+06 1.19e+06 1.19e+06
Dimensions without coordinates: Y, X, Y1, X1, Y2, X2, Y3, X3, Y4, X4
Data variables:
    0        (time, Y, X) float32 ...
    1        (time, Y1, X1) float32 ...
    2        (time, Y2, X2) float32 ...
    3        (time, Y3, X3) float32 ...
    4        (time, Y4, X4) float32 ...
Attributes: (12/14)
    multiscales:             [{'datasets': [{'path': '0'}, {'path': '1'}, {'p...
    OVR_RESAMPLING_ALG:      AVERAGE
    KeyDirectoryVersion:     1
    KeyRevision:             1
    KeyRevisionMinor:        0
    GTModelTypeGeoKey:       1
    ...                      ...
    GeogCitationGeoKey:      WGS 84
    GeogAngularUnitsGeoKey:  9102
    ProjectedCSTypeGeoKey:   32644
    ProjLinearUnitsGeoKey:   9001
    ModelPixelScale:         [20.0, 20.0, 0.0]
    ModelTiepoint:           [0.0, 0.0, 0.0, 399960.0, 1300020.0, 0.0]

In [ ]:
cds

In [ ]:
cds

In [ ]:
ds.coords["x"] = gen_xcoords(ds)

In [ ]:
# import dask
# import numpy as np
# import xarray as xr

# xr.set_options(display_style="html")

# def arbitrary_function(dataset):
#     return dataset["a"] * dataset["wavelength"] * dataset.attrs["wcs_param"]

# ds = xr.Dataset()
# # construct a dask array.
# # In practice this could represent an on-disk dataset,
# # with data reads only occurring when necessary
# ds["a"] = xr.DataArray(dask.array.ones((10,)), dims=["wavelength"], coords={"wavelength": np.arange(10)})

# # some coordinate system parameter
# ds.attrs["wcs_param"] = 1.0

# # complicated pixel to world function
# # no compute happens since we are working with dask arrays
# # so this is quite cheap.
# ds.coords["azimuth"] = arbitrary_function(ds)
# ds

In [ ]:
# dt = ds.xref.generate_multiscale_coords()

In [ ]:
# dt

In [ ]:
dt

In [ ]:
dt["1"].isel(time=0)["1"].plot(robust=True)

In [ ]:
dt["4"].isel(time=0)["4"].plot(robust=True)

In [ ]:
import hvplot.xarray
import xarray as xr

# xr_ds = xr.tutorial.open_dataset('air_temperature').load().sel(time='2013-06-01 12:00')
xr_ds = dt["4"].ds.isel(time=0)
xr_ds.hvplot()

In [ ]:
xr_ds = dt["1"].ds.isel(time=0)
xr_ds.hvplot()